In [1]:
import java.util.concurrent._
import language.implicitConversions

scala.language$@7f6add

In [2]:
// id : ex_7.1

class Par[A](ap : () => A) {
    
    val a : () => A = ap
        
    def get : A = a()
    
}

object Par {
    
    def unit[A](a : => A) : Par[A] = new Par(() => a)

    def map2[A,B,C](pa : Par[A], pb: Par[B])(f: (A, B) => C): Par[C] = {
        unit(f(pa.get,pb.get))
    }
    
}


$line19.$read$$iw$$iw$Par$@52232f30

In [3]:
def sum(ints: IndexedSeq[Int]): Par[Int] = 
    if (ints.size <= 1) Par.unit(ints.headOption getOrElse 0) 
    else {
        val (l,r) = ints.splitAt(ints.length/2)
        Par.map2(sum(l), sum(r))(_ + _)
    }

sum: (ints: IndexedSeq[Int])Par[Int]


In [4]:
sum(IndexedSeq(1,2,3,4,5)).get

15

In [5]:
// id : ex_7.2

// Shamelessly (or shamefully) taking the relevant portions from FP in Scala answers
type Par[A] = ExecutorService => Future[A]

object Par {

  def run[A](s: ExecutorService)(a: Par[A]): Future[A] = a(s)

  /*
  `unit` is represented as a function that returns a `UnitFuture`, which is a 
  simple implementation of `Future` that just wraps a constant value. It doesn't 
  use the `ExecutorService` at all. It's always done and can't be cancelled. Its 
  `get` method simply returns the value that we gave it.
  */
  def unit[A](a: A): Par[A] = (es: ExecutorService) => UnitFuture(a) 

  private case class UnitFuture[A](get: A) extends Future[A] {
    def isDone = true
    def get(timeout: Long, units: TimeUnit) = get
    def isCancelled = false
    def cancel(evenIfRunning: Boolean): Boolean = false
  }

  /*
  `map2` doesn't evaluate the call to `f` in a separate logical thread, in accord 
  with our design choice of having `fork` be the sole function in the API for 
  controlling parallelism. We can always do `fork(map2(a,b)(f))` if we want the 
  evaluation of `f` to occur in a separate thread.
  */
  def map2[A,B,C](a: Par[A], b: Par[B])(f: (A,B) => C): Par[C] = 
    (es: ExecutorService) => {
      val af = a(es)
      val bf = b(es)
      /* 
      This implementation of `map2` does _not_ respect timeouts. It simply passes 
      the `ExecutorService` on to both `Par` values, waits for the results of the 
      Futures `af` and `bf`, applies `f` to them, and wraps them in a `UnitFuture`. 
      In order to respect timeouts, we'd need a new `Future` implementation that 
      records the amount of time spent evaluating `af`, then subtracts that time 
      from the available time allocated for evaluating `bf`.
      */
      UnitFuture(f(af.get, bf.get)) 
    }

  /*
  This is the simplest and most natural implementation of `fork`, but there are 
  some problems with it--for one, the outer `Callable` will block waiting for the 
  "inner" task to complete. Since this blocking occupies a thread in our thread pool, 
  or whatever resource backs the `ExecutorService`, this implies that we're losing 
  out on some potential parallelism. Essentially, we're using two threads when one 
  should suffice. This is a symptom of a more serious problem with the implementation, 
  and we will discuss this later in the chapter.
  */
  def fork[A](a: => Par[A]): Par[A] = 
    es => es.submit(new Callable[A] {
      def call = a(es).get
    })

  def lazyUnit[A](a: => A): Par[A] = fork(unit(a))

}


$line22.$read$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$Par$@48cd3e36

In [6]:
// id : try ex_7.2

def sum(ints: IndexedSeq[Int]): Par[Int] = 
    if (ints.size <= 1) Par.unit(ints.headOption getOrElse 0) 
    else {
        val (l,r) = ints.splitAt(ints.length/2)
        Par.map2(sum(l), sum(r))(_ + _)
    }

val future = Par.run(Executors.newFixedThreadPool(1))(sum(IndexedSeq(1,2,3,4,5)))

println("future", future)
println("get", future.get)

(future,UnitFuture(15))
(get,15)


null

In [7]:
// id : ex_7.3
import java.util.concurrent.Future

type Par[A] = ExecutorService => Future[A]

object Par {

    def run[A](s: ExecutorService)(a: Par[A]): Future[A] = a(s)

    def unit[A](a: () => A): Par[A] = (es: ExecutorService) => es.submit(new Callable[A] {
          def call = a()
    })

    def map2[A,B,C](a: Par[A], b: Par[B])(f: (A,B) => C): Par[C] = 
        (es: ExecutorService) => {
        val af = a(es)
        val bf = b(es)
        Map2UnitFuture(af, bf, f) 
    }

    // https://github.com/fpinscala/fpinscala/blob/master/answerkey/parallelism/03.answer.scala
    
    case class Map2UnitFuture[A, B, C](
        a: Future[A], 
        b: Future[B],
        f : (A, B) => C) extends Future[C] {
        
        @volatile var cache: Option[C] = None
        def isDone = cache.isDefined
        def isCancelled = a.isCancelled || b.isCancelled
        def cancel(evenIfRunning: Boolean) =
            a.cancel(evenIfRunning) || b.cancel(evenIfRunning)
        def get = compute(Long.MaxValue)
        def get(timeout: Long, units: TimeUnit): C =
            compute(TimeUnit.NANOSECONDS.convert(timeout, units))

        private def compute(timeoutInNanos: Long): C = cache match {
            case Some(c) => c
            case None =>
              println("Timeout " + TimeUnit.SECONDS.convert(timeoutInNanos, TimeUnit.NANOSECONDS))
              val start = System.nanoTime
              val ar = a.get(timeoutInNanos, TimeUnit.NANOSECONDS)
              val stop = System.nanoTime;val aTime = stop-start
              println("Timeout " + TimeUnit.SECONDS.convert(timeoutInNanos - aTime, TimeUnit.NANOSECONDS))
              val br = b.get(timeoutInNanos - aTime, TimeUnit.NANOSECONDS)
              val ret = f(ar, br)
              cache = Some(ret)
              ret
        }
    }

}

$line24.$read$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$Par$@17b931ea

In [8]:
def sum(ints: IndexedSeq[Int]): Par[Int] = 
    if (ints.size <= 1) {
        ints match {
            case IndexedSeq(x) => Par.unit(() => x)
            case IndexedSeq() => Par.unit(() => 0)
        }
    } else {
        val (l,r) = ints.splitAt(ints.length/2)
        Par.map2(sum(l), sum(r))({
            Thread.sleep(1000)
            _ + _
        })
    }


sum: (ints: IndexedSeq[Int])Par[Int]


In [9]:
val future = Par.run(Executors.newFixedThreadPool(4))(sum(IndexedSeq(1,2,3,4,5,6,7,8,9,10)))

Map2UnitFuture(Map2UnitFuture(Map2UnitFuture(java.util.concurrent.FutureTask@382f6e5d,java.util.concurrent.FutureTask@4fcb6e3a,<function2>),Map2UnitFuture(java.util.concurrent.FutureTask@1fee39d7,Map2UnitFuture(java.util.concurrent.FutureTask@3fe31189,java.util.concurrent.FutureTask@7cfdddae,<function2>),<function2>),<function2>),Map2UnitFuture(Map2UnitFuture(java.util.concurrent.FutureTask@68d3c6b5,java.util.concurrent.FutureTask@69d6af9,<function2>),Map2UnitFuture(java.util.concurrent.FutureTask@21555cfa,Map2UnitFuture(java.util.concurrent.FutureTask@3357f5d3,java.util.concurrent.FutureTask@45476b62,<function2>),<function2>),<function2>),<function2>)

In [10]:
future.get(1, TimeUnit.NANOSECONDS)

Timeout 0
Timeout 0
Timeout 0
Timeout 0
Timeout 0
Timeout 0
Timeout 0
Timeout 0
Timeout 0
Timeout 0
Timeout 0
Timeout 0
Timeout 0
Timeout 0
Timeout 0
Timeout 0
Timeout 0
Timeout 0


55

In [11]:
val sum2 = Par.map2(Par.unit(() => { Thread.sleep(10000); 1}), 
                    Par.unit(() => { Thread.sleep(10000); 2}))((a: Int, b: Int) => a + b)
sum2(Executors.newFixedThreadPool(4)).get(0, TimeUnit.NANOSECONDS)

Timeout 0


java.util.concurrent.TimeoutException: java.util.concurrent.TimeoutException

In [12]:
val sum2 = Par.map2(Par.unit(() => { Thread.sleep(10000); 1}), 
                    Par.unit(() => { Thread.sleep(30000); 2}))((a: Int, b: Int) => a + b)
sum2(Executors.newFixedThreadPool(4)).get(11, TimeUnit.SECONDS)

Timeout 11
Timeout 1


java.util.concurrent.TimeoutException: java.util.concurrent.TimeoutException

In [13]:
val sum2 = Par.map2(Par.unit(() => { Thread.sleep(2500); 1}), 
                    Par.unit(() => { Thread.sleep(2500); 2}))((a: Int, b: Int) => a + b)
Par.run(Executors.newFixedThreadPool(4))(sum2).get(3, TimeUnit.SECONDS)

Timeout 3
Timeout 0


3